In [1]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Training').getOrCreate()

24/04/04 22:13:02 WARN Utils: Your hostname, Koyanas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.17 instead (on interface en0)
24/04/04 22:13:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 22:13:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
tr=spark.read.csv('test_ml.csv',header=True,inferSchema=True)
tr.show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
| Rahul| 19|  5000|         0|
|Shweta| 20|  9000|         0|
| Mohit| 22| 16000|         1|
|  Raju| 25| 12000|         1|
|Preeti| 30| 25000|         2|
|Laxman| 24| 32000|         2|
|Keshav| 21| 17000|         1|
+------+---+------+----------+



In [3]:
tr.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [4]:
from pyspark.ml.feature import VectorAssembler
featureass=VectorAssembler(inputCols=['Age','Experience'],outputCol="new_feature")

In [5]:
df=featureass.transform(tr)

In [6]:
df.show()

+------+---+------+----------+-----------+
|  Name|Age|Salary|Experience|new_feature|
+------+---+------+----------+-----------+
| Rahul| 19|  5000|         0| [19.0,0.0]|
|Shweta| 20|  9000|         0| [20.0,0.0]|
| Mohit| 22| 16000|         1| [22.0,1.0]|
|  Raju| 25| 12000|         1| [25.0,1.0]|
|Preeti| 30| 25000|         2| [30.0,2.0]|
|Laxman| 24| 32000|         2| [24.0,2.0]|
|Keshav| 21| 17000|         1| [21.0,1.0]|
+------+---+------+----------+-----------+



In [7]:
data=df.select(['new_feature','Salary'])
data.show()

+-----------+------+
|new_feature|Salary|
+-----------+------+
| [19.0,0.0]|  5000|
| [20.0,0.0]|  9000|
| [22.0,1.0]| 16000|
| [25.0,1.0]| 12000|
| [30.0,2.0]| 25000|
| [24.0,2.0]| 32000|
| [21.0,1.0]| 17000|
+-----------+------+



In [8]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='new_feature',labelCol='Salary')
regressor=regressor.fit(train_data)

24/04/04 22:26:34 WARN Instrumentation: [c6559e35] regParam is zero, which might cause numerical instability and overfitting.
24/04/04 22:26:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/04 22:26:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [9]:
regressor.coefficients

DenseVector([-1200.0, 18700.0])

In [10]:
regressor.intercept

23500.000000000062

In [11]:
result=regressor.evaluate(test_data)

In [12]:
result.predictions.show()

+-----------+------+-------------------+
|new_feature|Salary|         prediction|
+-----------+------+-------------------+
| [19.0,0.0]|  5000|  700.0000000000873|
| [20.0,0.0]|  9000|-499.99999999990905|
+-----------+------+-------------------+

